In [1]:

import pandas as pd
from pymongo import MongoClient
df=pd.read_csv("netflix_titles.csv")
df=df.where(pd.notnull(df),None)


In [2]:
Client=MongoClient("mongodb://localhost:27017/")
db=Client["netflix_db"]
collection =db["titles"]
data = df.to_dict(orient="records")
collection.insert_many(data)
print("data inserted into mongodb sucessfully")

data inserted into mongodb sucessfully


In [3]:
cursor = collection.find()
df=pd.DataFrame(list(cursor))
df.drop(columns=['_id'],inplace=True)
print(df.head())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water             None   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans             None   
4      s5  TV Show           Kota Factory             None   

                                                cast        country  \
0                                               None  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...           None   
3                                               None           None   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [4]:
df['date_added'] = pd.to_datetime(df['date_added'],
errors='coerce')
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

In [5]:
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

In [6]:
df[['duration_int', 'duration_type']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')
df['duration_int'] = pd.to_numeric(df['duration_int'], errors='coerce')

In [7]:
df.fillna({
    'director': 'Unknown',
    'cast': 'Not specified',
    'country': 'Unknown',
    'rating': 'Not Rated'
}, inplace=True)

In [8]:
df.to_csv("netflix_cleaned.csv", index=False)
print("✅ Data cleaned and saved as 'netflix_cleaned1.csv'")

✅ Data cleaned and saved as 'netflix_cleaned1.csv'


In [ ]:
import pandas as pd

# Load cleaned CSV
df_cleaned = pd.read_csv("netflix_cleaned.csv")

# Fill NaNs with None (MongoDB can't store NaN)
df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["netflix_db"]

# OPTIONAL: Drop old cleaned collection if it exists (to avoid duplicates)
db.drop_collection("titles_cleaned")

# Create a new collection
collection_cleaned = db["titles_cleaned"]

In [ ]:
# Convert DataFrame to dictionary
data_dict = df_cleaned.to_dict(orient='records')

# Insert into MongoDB
collection_cleaned.insert_many(data_dict)

print("✅ Cleaned data re-uploaded to MongoDB successfully!")

✅ Cleaned data re-uploaded to MongoDB successfully!


In [2]:
import pandas as pd

df = pd.read_csv("netflix_titles.csv")  # or the correct file name

In [3]:
df['combined'] = df['title'] + ' ' + df['description'] + ' ' + df['listed_in'] + ' ' + df['cast']
df['combined'] = df['combined'].fillna('')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined'])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(df.index, index=df['title'].str.lower())

def get_recommendations(title, cosine_sim=cosine_sim):
    title = title.lower()
    if title not in indices:
        return "Title not found!"
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [8]:
get_recommendations("Narcos")

3298           Wild District
2921          Narcos: Mexico
6672               El Cartel
1268    El final del paraíso
2134         The Great Heist
Name: title, dtype: object

In [9]:
get_recommendations("Ganglands")

2668                      Earth and Blood
3976               The Eagle of El-Se'eed
3425                          Street Flow
4399                              Warrior
697     Elite Short Stories: Carla Samuel
Name: title, dtype: object